In [1]:
import pandas as pd
import sklearn

In [38]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PowerTransformer

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.metrics import root_mean_squared_error, r2_score

In [12]:
data = pd.read_csv("test_merge_data.csv", index_col=0)
data.head()

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp,price_improvement
1,ID86355,2025-09-10 09:30:05.000,2025-09-10 09:30:05.103,MWYN,1,225,2.65,2.65,ID1516,MWYN,2.61,2.65,5.0,2.0,11.0,8.0,2025-09-10 09:30:04.181625568,0.0
2,ID86359,2025-09-10 09:30:05.962,2025-09-10 09:33:37.670,CDTG,1,1000,1.30,1.30,ID1516,CDTG,1.30,1.31,45.0,19.0,8.0,11.0,2025-09-10 09:30:05.736571594,0.0
4,ID86369,2025-09-10 09:30:07.228,2025-09-10 09:37:43.187,AEHL,1,100,12.00,12.00,ID1516,AEHL,13.33,13.64,1.0,1.0,8.0,11.0,2025-09-10 09:30:06.628618340,0.0
5,ID86382,2025-09-10 09:30:10.105,2025-09-10 09:30:10.210,CUPR,2,89,2.93,3.03,ID1516,CUPR,3.03,3.03,1.0,3.0,12.0,11.0,2025-09-10 09:30:10.091706974,0.1
6,ID86390,2025-09-10 09:30:11.890,2025-09-10 09:30:25.795,CUPR,2,53,3.05,3.05,ID1516,CUPR,3.02,3.03,9.0,1.0,11.0,11.0,2025-09-10 09:30:11.866494533,0.0


In [13]:
data.reset_index(inplace=True)
data.drop(["index"], axis=1, inplace=True)
data.head()

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp,price_improvement
0,ID86355,2025-09-10 09:30:05.000,2025-09-10 09:30:05.103,MWYN,1,225,2.65,2.65,ID1516,MWYN,2.61,2.65,5.0,2.0,11.0,8.0,2025-09-10 09:30:04.181625568,0.0
1,ID86359,2025-09-10 09:30:05.962,2025-09-10 09:33:37.670,CDTG,1,1000,1.30,1.30,ID1516,CDTG,1.30,1.31,45.0,19.0,8.0,11.0,2025-09-10 09:30:05.736571594,0.0
2,ID86369,2025-09-10 09:30:07.228,2025-09-10 09:37:43.187,AEHL,1,100,12.00,12.00,ID1516,AEHL,13.33,13.64,1.0,1.0,8.0,11.0,2025-09-10 09:30:06.628618340,0.0
3,ID86382,2025-09-10 09:30:10.105,2025-09-10 09:30:10.210,CUPR,2,89,2.93,3.03,ID1516,CUPR,3.03,3.03,1.0,3.0,12.0,11.0,2025-09-10 09:30:10.091706974,0.1
4,ID86390,2025-09-10 09:30:11.890,2025-09-10 09:30:25.795,CUPR,2,53,3.05,3.05,ID1516,CUPR,3.02,3.03,9.0,1.0,11.0,11.0,2025-09-10 09:30:11.866494533,0.0


In [15]:
data.groupby("exchange").count()

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp,price_improvement
exchange,,,,,,,,,,,,,,,,,
ID1516,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179
ID245333,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
ID295386,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
ID29608,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60
ID412967,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [16]:
ex1 = data.loc[data.exchange == "ID1516"]
ex2 = data.loc[data.exchange == "ID29608"]

In [24]:
#side automatically acts like a dummy variable for side=="B" !
VARIABLES = ["side", "order_qty", "limit_price", "bid_price", "ask_price", "bid_size", "ask_size"]

X = ex1[VARIABLES]
y = ex1["price_improvement"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [26]:
#use linear regression as a baseline:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
preds = model.predict(X_test_scaled)
rmse = root_mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print([rmse, r2])



[0.1314296793491806, 0.028539489096729542]


In [27]:
#try adding a power transformer

pt = PowerTransformer(method="yeo-johnson", standardize=False)
pt.fit(X_train)
X_train_pt = pt.transform(X_train)
X_test_pt  = pt.transform(X_test)

scaler = StandardScaler()
scaler.fit(X_train_pt)
X_train_preprocessed = scaler.transform(X_train_pt)
X_test_preprocessed  = scaler.transform(X_test_pt)

model = LinearRegression()
model.fit(X_train_preprocessed, y_train)
preds = model.predict(X_test_preprocessed)
rmse = root_mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print([rmse, r2])

[0.13168482792063216, 0.024763974181401527]


In [ ]:
#do a quick temp check on the different models:

#power transformer didn't help (only hurt) so we will not use it

models = {"LinearRegression": LinearRegression(),
          "Ridge": Ridge(), "Lasso": Lasso(),
          "ElasticNet": ElasticNet(),
          "RandomForest": RandomForestRegressor(),
          "ExtraTrees": ExtraTreesRegressor(),
          "GradientBoosting": GradientBoostingRegressor()}

results = {}

for model_name in models:
    model = models[model_name]
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[model_name] = [rmse, r2]

In [42]:
pd.DataFrame(results, index=["RMSE", "R2"]).T

,RMSE,R2
LinearRegression,0.131430,0.028539
Ridge,0.132500,0.012652
Lasso,0.133349,-0.000046
ElasticNet,0.133349,-0.000046
RandomForest,0.126843,0.095163
ExtraTrees,0.112380,0.289736
GBR,0.134626,-0.019283


In [49]:
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "RandomForest": RandomForestRegressor(n_jobs=-1),
    "ExtraTrees": ExtraTreesRegressor(n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(),
}

param_grids = {
    "LinearRegression": {
        "fit_intercept": [True, False],
    },

    "Ridge": {
        "alpha": [0.1, 1.0, 10.0],
    },

    "Lasso": {
        "alpha": [0.001, 0.01, 0.1],
        "max_iter": [5000],
    },

    "ElasticNet": {
        "alpha": [0.001, 0.01, 0.1],
        "l1_ratio": [0.2, 0.5, 0.8],
        "max_iter": [5000],
    },

    "RandomForest": {
        "n_estimators": [100, 300],
        "max_depth": [None, 10, 30],
        "min_samples_leaf": [1, 3],
        "max_features": ["sqrt", "log2"],
    },

    "ExtraTrees": {
        "n_estimators": [100, 300],
        "max_depth": [None, 10, 30],
        "min_samples_leaf": [1, 3],
        "max_features": ["sqrt", "log2"],
    },

    "GradientBoosting": {
        "n_estimators": [100, 300],
        "learning_rate": [0.05, 0.1],
        "max_depth": [2, 3],
    },

    "SVR": {
        "C": [1, 10],
        "gamma": ["scale"],
        "kernel": ["rbf"],
    },

    "GaussianProcess": {
        "alpha": [1e-6, 1e-3],
    }
}

results = {}

for name, model in models.items():
    print(f"Grid search for {name}...")

    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        cv=5,
        scoring="neg_root_mean_squared_error",
    )

    gs.fit(X_train_scaled, y_train)

    best_model = gs.best_estimator_
    preds = best_model.predict(X_test_scaled)

    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    results[name] = {
        "best_params": gs.best_params_,
        "rmse": rmse,
        "r2": r2,
    }


Grid search for LinearRegression...
Grid search for Ridge...
Grid search for Lasso...
Grid search for ElasticNet...
Grid search for RandomForest...


KeyboardInterrupt: 

In [46]:
pd.DataFrame(results).T

,best_params,rmse,r2
LinearRegression,{'fit_intercept': True},0.13143,0.028539
Ridge,{'alpha': 1.0},0.1325,0.012652
Lasso,"{'alpha': 0.001, 'max_iter': 5000}",0.133039,0.004607
ElasticNet,"{'alpha': 0.001, 'l1_ratio': 0.8, 'max_iter': ...",0.133039,0.004607
RandomForest,"{'max_depth': None, 'max_features': 'log2', 'm...",0.124663,0.125996
ExtraTrees,"{'max_depth': None, 'max_features': 'sqrt', 'm...",0.109485,0.325865
GBR,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.136047,-0.040924


In [50]:
X = ex2[VARIABLES]
y = ex2["price_improvement"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [51]:
#use linear regression as a baseline:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
preds = model.predict(X_test_scaled)
rmse = root_mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print([rmse, r2])



[0.1878890224280487, -0.230014941539884]


In [52]:
#try adding a power transformer

pt = PowerTransformer(method="yeo-johnson", standardize=False)
pt.fit(X_train)
X_train_pt = pt.transform(X_train)
X_test_pt  = pt.transform(X_test)

scaler = StandardScaler()
scaler.fit(X_train_pt)
X_train_preprocessed = scaler.transform(X_train_pt)
X_test_preprocessed  = scaler.transform(X_test_pt)

model = LinearRegression()
model.fit(X_train_preprocessed, y_train)
preds = model.predict(X_test_preprocessed)
rmse = root_mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print([rmse, r2])

[0.18561804251400593, -0.20046070891193657]


In [53]:
#since this is a very small dataset (n=60), we will include some models that are more time intensive
#but work well for small datasets.

#in this case the power transformer helped a bit so we will keep it.

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "RandomForest": RandomForestRegressor(n_jobs=-1),
    "ExtraTrees": ExtraTreesRegressor(n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(),
    "SVR": SVR(),
    "GaussianProcess": GaussianProcessRegressor()
}

results = {}

for name, model in models.items():
    print(f"Grid search for {name}...")

    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        cv=5,
        scoring="neg_root_mean_squared_error",
    )

    gs.fit(X_train_preprocessed, y_train)

    best_model = gs.best_estimator_
    preds = best_model.predict(X_test_preprocessed)

    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    results[name] = {
        "best_params": gs.best_params_,
        "rmse": rmse,
        "r2": r2,
    }


Grid search for LinearRegression...
Grid search for Ridge...
Grid search for Lasso...
Grid search for ElasticNet...
Grid search for RandomForest...
Grid search for ExtraTrees...
Grid search for GradientBoosting...
Grid search for SVR...
Grid search for GaussianProcess...


In [54]:
pd.DataFrame(results).T

,best_params,rmse,r2
LinearRegression,{'fit_intercept': True},0.185618,-0.200461
Ridge,{'alpha': 10.0},0.185357,-0.197087
Lasso,"{'alpha': 0.01, 'max_iter': 5000}",0.183745,-0.176351
ElasticNet,"{'alpha': 0.01, 'l1_ratio': 0.8, 'max_iter': 5...",0.183745,-0.176351
RandomForest,"{'max_depth': None, 'max_features': 'sqrt', 'm...",0.184939,-0.191691
ExtraTrees,"{'max_depth': None, 'max_features': 'sqrt', 'm...",0.185067,-0.193349
GradientBoosting,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.18588,-0.203856
SVR,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",0.169964,-0.006524
GaussianProcess,{'alpha': 0.001},0.187199,-0.220998


In [ ]:
#SVR is the most promising, let's run with it and find the right kernel

#since our dataset is very very small, we will switch to using leave-one-out cross validation in the grid search

from sklearn.model_selection import LeaveOneOut

svr = SVR()

param_grid = [
        # ----- RBF kernel -----
        {
            "kernel": ["rbf"],
            "C": [0.1, 1, 10],
            "gamma": ["scale", "auto"],
            "epsilon": [0.01, 0.1],
        },
        # ----- Linear kernel -----
        {
            "kernel": ["linear"],
            "C": [0.1, 1, 10],
            "epsilon": [0.01, 0.1],
        },
        # ----- Polynomial kernel -----
        {
            "kernel": ["poly"],
            "degree": [2, 3],
            "C": [0.1, 1],
            "gamma": ["scale", "auto"],
            "epsilon": [0.01, 0.1],
            "coef0": [0.0, 1.0],
        },
        # ----- Sigmoid kernel -----
        {
            "kernel": ["sigmoid"],
            "C": [0.1, 1, 10],
            "gamma": ["scale", "auto"],
            "epsilon": [0.01, 0.1],
            "coef0": [0.0, 1.0],
        },
    ]


cv = LeaveOneOut()

gs_svr = GridSearchCV(
    estimator=svr,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=cv,
    n_jobs=1,
)

gs_svr.fit(X_train_preprocessed, y_train)

print("Best params:", gs_svr.best_params_)
print("Best CV RMSE:", -gs_svr.best_score_)


TypeError: unhashable type: 'list'

In [59]:
best_model = gs_svr.best_estimator_
preds = best_model.predict(X_test_preprocessed)
print(root_mean_squared_error(y_test, preds), r2_score(y_test, preds))

0.18564615153657854 -0.20082431938533163


In [ ]:
#so it would appear we over fit the model. let's revert back to the old one.

